
# Import Dependencies 

In [1]:
# COCO related libraries 
from samples.coco import coco

#MaskRCNN libraries 
from mrcnn.config import Config
import mrcnn.utils as utils 
from mrcnn import visualize 
import mrcnn.model as modellib
from mrcnn.model import MaskRCNN
from keras.models import load_model
from mrcnn.visualize import display_images
from mrcnn.model import log


# Misc
import os
import sys 
import json
import numpy as np
import time 
from PIL import Image, ImageDraw
import imgaug 
from imgaug import augmenters

C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (


# Constants 

In [9]:
# Number of classes in dataset
NUM_CLASSES = 2

# Relative path to .h5 weights file 
WEIGHTS_FILE = None

# Relative path to train annotations JSON file
TRAIN_ANNOTATIONS_FILE = "C:/Users/FATTY/MaskRCNN/myMaskRCNN/Train_images/annotations.json"

#  Relative path to directory of tRAIN images 
TRAIN_ANNOTATIONS_IMAGE_DIR = "C:/Users/FATTY/MaskRCNN/myMaskRCNN/Train_images"


# Relative path to validation annotations JSON file
VALIDATION_ANNOTATIONS_FILE = "C:/Users/FATTY/MaskRCNN/myMaskRCNN/Val_images/annotations.json"


#  Relative path to directory of VALIDATION images 
VALIDATION_ANNOTATION_IMAGE_DIR = "C:/Users/FATTY/MaskRCNN/myMaskRCNN/Val_images"


# Number of epochs to train on 
NUM_EPOCHS = 100

MODEL_NAME = "model_1"



#  Additional setup

In [10]:
# Set the root directory to the root directiory of MASK RCNN git repo
ROOT_DIR = os.getcwd()

# Directory to save trained models and logs 
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# select which GPU to use 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0";


# Declare Training Configurations 

In [11]:
class myMaskRCNNConfig(coco.CocoConfig):
    """
    """
    #Give the configuration a recognizable name 
    NAME = "MaskRCNN_config"
    
    # Train on 1 image per GPU 
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 

    # Number of class including background 
    NUM_CLASSES = 1 + 1

    # Min and Max image dimensions
    IMAGE_MIN_DIM = 1152
    IMAGE_MIN_DIM = 1280

    # You can experiment with this number to see if it improves training 
    STEPS_PER_EPOCH = 80

    # This is how often validation is run, if you are using too much hard drive space on saved models try to make this value larger
    VALIDATION_STEPS = 100 

    # Matterport use resnet101, here we downsize to fit our graphic card 
    BACKBONE = "resnet101" 


    # RPN ANCHOR SCALES 
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)

    # changed to 512 because that's the number use in the original MaskRCNN paper 
    
    TRAIN_ROIS_PER_IMAGE = 200
    MAX_GT_INSTANCES = 114
    POST_NMS_ROIS_INFERENCE = 1000
    POST_NMS_ROIS_TRAINING = 2000

    DETECTION_MAX_INSTANCES = 125
    DETECTION_MIN_CONFIDENCE = 0.8
    
    MINI_MASK_SHAPE = (128, 128)
    MAX_GT_INSTANCES = 125



# Create an instance of the myMaskRCNNConfig class

In [12]:
config = myMaskRCNNConfig()


# Let's display all config values 

In [13]:
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        125
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1280
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 10.0, 'rpn_bbox_loss': 10.0, 'mrcnn_class_loss': 10.0, 'mrcnn_bbox_loss': 10.0, 'mrcnn_mask_loss': 10.0}
MASK_POOL_SIZE                 14
MASK_SHAPE   

 
 # Create class to load dataset

In [14]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """

    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        
        
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids


# Load train and validation datasets 

In [15]:
dataset_train = CocoLikeDataset()
dataset_train.load_data(TRAIN_ANNOTATIONS_FILE, TRAIN_ANNOTATIONS_IMAGE_DIR)
dataset_train.prepare()

dataset_val = CocoLikeDataset()
dataset_val.load_data(VALIDATION_ANNOTATIONS_FILE, VALIDATION_ANNOTATION_IMAGE_DIR)
dataset_val.prepare()

# Build MaskRCNN Model

In [16]:
# Create model in training model
model = modellib.MaskRCNN(mode = "training", config = myMaskRCNNConfig() , model_dir = MODEL_DIR)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


# Load weights into model if weights file is not None 
This is meant to be used if you are refining on a set of preexisting weights 

In [17]:
if WEIGHTS_FILE is not None:
    model.load_weights(WEIGHTS_FILE, by_name = True)

# Train model 

The model after each epoch will be saved in the logs folder 

In [18]:
start_train = time.time()
model.train(dataset_train, dataset_val, learning_rate = myMaskRCNNConfig().LEARNING_RATE, epochs = NUM_EPOCHS, layers = 'all',
           augmentation = imgaug.augmenters.Sequential([
                imgaug.augmenters.Fliplr(1),
                imgaug.augmenters.Flipud(1),
                imgaug.augmenters.Affine(rotate=(-45, 45)),
                imgaug.augmenters.Affine(rotate=(-90, 90)),
                imgaug.augmenters.LinearContrast((0.75, 1.5)),
                imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
                imgaug.augmenters.Affine(scale=(0.5, 1.5))
            ], random_order=True # apply augmenters in random order
            ))
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\FATTY\MaskRCNN\myMaskRCNN\logs\maskrcnn_config20200728T1415\mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branc

C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\FATTY\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch 1/100
 5/80 [>.............................] - ETA: 47:04 - loss: 239.1381 - rpn_class_loss: 91.1920 - rpn_bbox_loss: 123.5848 - mrcnn_class_loss: 3.8298 - mrcnn_bbox_loss: 16.0803 - mrcnn_mask_loss: 4.4512            

ResourceExhaustedError: OOM when allocating tensor with shape[1,65536,2304] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/SGD/gradients/rpn_model/rpn_conv_shared/convolution_grad/Conv2DBackpropFilter}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
